In [1]:
import pandas as pd
fp = "../../data/retail_q1_post_EDA.parquet"
df = pd.read_parquet(fp)

In [3]:
df["item_total"] = df["Quantity"] * df["Price"]
dsbysc = df.groupby([df.InvoiceDate.dt.day_of_year, df.StockCode])
dsbysc = dsbysc["item_total"].sum().to_frame().reset_index()
dfQ1_PA = dsbysc.pivot(index="InvoiceDate", columns="StockCode", values="item_total").fillna(0)

In [5]:
fp = "../../data/retail_q1_post_data_rep_prep.parquet"
dfQ1_PA.to_parquet(fp, index=False)

In [6]:
from ontology.kmds_ontology import *
from tagging.tag_types import ExploratoryTags
from owlready2 import *
from utils.path_utils import get_kb_file_path, get_ontology_path
KNOWLEDGE_BASE = "example_ml_kb_exp_workflow"
onto = get_ontology(get_kb_file_path(KNOWLEDGE_BASE)).load()